In [ ]:
import numpy as np, os
from glob import glob
from tqdm.auto import tqdm
from zhh import parse_sample_path, get_preselection_passes, ProcessIndex

DATA_ROOT = f'/nfs/dust/ilc/user/bliewert/zhh'
INDEX_DIR = '/nfs/dust/ilc/user/bliewert/zhh/CreateRawIndex/v1'

processes = np.load(f"{INDEX_DIR if os.name != 'nt' else REPO_ROOT}/processes.npy")
samples = np.load(f"{INDEX_DIR if os.name != 'nt' else REPO_ROOT}/samples.npy")
chunks = np.load(f'{DATA_ROOT}/CreatePreselectionChunks/v1/chunks.npy') if os.name != 'nt' else np.load(f'{REPO_ROOT}/chunks_f.npy')

Validate chunks

In [ ]:
subset = chunks_f[chunks_f['chunk_size_factual'] <= 0]
if len(subset) > 0:
    raise Exception(f"Invalid chunk length mismatch for branch <{subset['branch']}>")
else:
    print('Chunks OK')

Introduce indices on samples and processes (included in future runs)

In [ ]:
dtype_sample = ProcessIndex.dtype_sample
dtype_process = ProcessIndex.dtype_process

if not 'sid' in samples.dtype.names:
    samples_new = np.empty(len(samples), dtype=dtype_sample)
    samples_new['sid'] = np.arange(len(samples))
    for col in samples.dtype.names:
        samples_new[col] = samples[col]
        
    if len(samples) == len(samples_new):
        np.save(f'{INDEX_DIR}/samples.npy', samples_new)

if not 'pid' in processes.dtype.names:
    #processes_new = np.array([pids, *(processes[col] for col in processes.dtype.names)], dtype=dtype_process)
    processes_new = np.empty(len(processes), dtype=dtype_process)
    processes_new['pid'] = np.arange(len(processes))
    for col in processes.dtype.names:
        processes_new[col] = processes[col]

    if len(processes) == len(processes_new):
        np.save(f'{INDEX_DIR}/processes.npy', processes_new)
        
print('Conversion successful')

Prototyping

In [ ]:
from ast import literal_eval as make_tuple
import json

def test_meta_files(DATA_ROOT:str=f'{DATA_ROOT}/PreselectionFinal/{version}')->bool:
    files = glob(f'{DATA_ROOT}/*_Source.txt')
    #branches = list(map(lambda x: x.split(f'{DATA_ROOT}/')[1].split('_Source.txt')[0], files))

    for f in tqdm(files):
        branch = f.split(f'{DATA_ROOT}/')[1].split('_Source.txt')[0]
        
        if osp.isfile(f'{DATA_ROOT}/{branch}_FinalStateMeta.json'):
            with open(f, 'r') as file:
                src_spec = file.read()
                if src_spec.startswith('('):
                    src_file, chunk_start, chunk_end = make_tuple(src_spec)
                else:
                    src_file = src_spec
            
            # Read metadata
            with open(f'{DATA_ROOT}/{branch}_FinalStateMeta.json', 'r') as file:
                meta = json.load(file)

            n_gen = meta['nEvtSum']
            proc = meta["processName"]
            
            if proc == '' or n_gen == 0:
                print(src_file)
                raise Exception(branch)
    
    return True

test_meta_files()

Delete invalid branches

In [ ]:
for branch in chunks_f['branch'][chunks_f['branch'] >= 15497]:
    post_files_suffixes = ['_PreSelection_llHH.root', '_PreSelection_vvHH.root', '_PreSelection_qqHH.root', '_FinalStates.root', '_FinalStateMeta.json', '.slcio', '_Source.txt']
    
    for suffix in post_files_suffixes:
        pass
        #os.remove(f'{DATA_ROOT}/PreselectionFinal/v1/{branch}{suffix}')